In [14]:
import pandas as pd
import numpy as np

In [15]:
from sklearn.cluster import MiniBatchKMeans

def get_featured_df(path_to_big_table_csv, 
                    path_to_weather_csv,
                    is_train):
    big_table_df = pd.read_csv(path_to_big_table_csv)
    
    coords = np.vstack(
        (big_table_df[['lat_start', 'lon_start']].values,
         big_table_df[['lat_end', 'lon_end']].values)
    )
    sample_indices = np.random.permutation(len(coords))
    kmeans = MiniBatchKMeans(n_clusters=90, batch_size=10000).fit(coords[sample_indices])
    
    big_table_df.loc[:, 'cluster_start'] = kmeans.predict(big_table_df[['lat_start', 'lon_start']])
    big_table_df.loc[:, 'cluster_end'] = kmeans.predict(big_table_df[['lat_end', 'lon_end']])
    
    big_table_df.loc[:, 'is_cluster_loaded_start'] = [1 if (i >= 50) else 0 for i in big_table_df['cluster_start']]
    big_table_df.loc[:, 'is_cluster_loaded_end'] = [1 if (i >= 50) else 0 for i in big_table_df['cluster_end']]
        
    weather_df = pd.read_csv(path_to_weather_csv)
    weather_df = weather_df[["date", "time", "humidity", "visibility", "windspeedKmph", "tempC", "weatherStatus"]]
    
    big_table_df['running_time'] = pd.to_datetime(big_table_df['running_time'])
    if is_train:
        big_table_df['completed_time'] = pd.to_datetime(big_table_df['completed_time'])
    
    big_table_df["is_work_hour"] = [1 if (i >= 8 and i <= 18) else 0 for i in big_table_df['running_time'].dt.hour]
    big_table_df['is_night_time'] = [1 if (i >= 19 and i <= 23) else 0 for i in big_table_df['running_time'].dt.hour]
    big_table_df['is_late_night_time'] = [1 if (i >= 0 and i <= 5) else 0 for i in big_table_df['running_time'].dt.hour]
    big_table_df['is_rush_hour'] = [1 if ((i >= 6 and i <= 9) or (i >= 17 and i <= 20)) else 0 for i in big_table_df['running_time'].dt.hour]
    
    big_table_df['date'] = big_table_df['running_time'].dt.date
    big_table_df['time'] = big_table_df['running_time']
    big_table_df['time'] = pd.to_numeric(big_table_df['time'].dt.strftime('%H')) * 100
    
    weather_df['date'] = pd.to_datetime(weather_df['date'], infer_datetime_format=True).dt.date
    
    big_table_df = big_table_df.merge(weather_df, how='inner', on=['date','time'])
    big_table_df.drop(["date", "time"], axis=1, inplace=True)
    
    return big_table_df

In [16]:
big_table_train_df = get_featured_df(path_to_big_table_csv="datasets/big-table-train.csv", 
                                     path_to_weather_csv="datasets/csv/weather.csv",
                                     is_train=True)
big_table_train_df.head()

/Users/dmytro/miniforge3/envs/ai_house_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/dmytro/miniforge3/envs/ai_house_env/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but MiniBatchKMeans was fitted without feature names
  warnings.warn(
/Users/dmytro/miniforge3/envs/ai_house_env/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but MiniBatchKMeans was fitted without feature names
  warnings.warn(


,Id,running_time,completed_time,route_distance_km,delta_time,node_start,node_finish,distance,speed,lon_start,...,is_cluster_loaded_end,is_work_hour,is_night_time,is_late_night_time,is_rush_hour,humidity,visibility,windspeedKmph,tempC,weatherStatus
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980417,4768348185,31.771489,30.0,30.738275,...,16,1,0,0,1,87,10,18,-4,Light snow
1,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980418,1570776947,3.673054,24.0,0.000000,...,0,1,0,0,1,87,10,18,-4,Light snow
2,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980430,4768348188,15.550612,27.0,0.000000,...,0,1,0,0,1,87,10,18,-4,Light snow
3,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980431,5125489557,50.034390,29.0,30.739016,...,79,1,0,0,1,87,10,18,-4,Light snow
4,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980432,5213300015,13.453126,30.0,30.739248,...,79,1,0,0,1,87,10,18,-4,Light snow


In [17]:
big_table_train_df.to_csv("datasets/big-table-train.csv", index=False)

In [18]:
big_table_test_df = get_featured_df(path_to_big_table_csv="datasets/big-table-test.csv", 
                                    path_to_weather_csv="datasets/csv/weather.csv", 
                                    is_train=False)
big_table_test_df.head()

/Users/dmytro/miniforge3/envs/ai_house_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/dmytro/miniforge3/envs/ai_house_env/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but MiniBatchKMeans was fitted without feature names
  warnings.warn(
/Users/dmytro/miniforge3/envs/ai_house_env/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but MiniBatchKMeans was fitted without feature names
  warnings.warn(


,Id,running_time,route_distance_km,node_start,node_finish,distance,speed,lon_start,lat_start,lon_end,...,is_cluster_loaded_end,is_work_hour,is_night_time,is_late_night_time,is_rush_hour,humidity,visibility,windspeedKmph,tempC,weatherStatus
0,6198,2022-01-24 03:38:30,4.744,8952394129,8952394128,138.795710,37.0,0.000000,0.000000,0.000000,...,0,0,0,1,0,84,10,24,-5,Clear
1,6198,2022-01-24 03:38:30,4.744,2059503754,4548172320,95.273001,33.0,30.724095,46.431127,30.724951,...,76,0,0,1,0,84,10,24,-5,Clear
2,6198,2022-01-24 03:38:30,4.744,2059504508,2059503754,137.647881,35.0,30.722883,46.430222,30.724095,...,76,0,0,1,0,84,10,24,-5,Clear
3,6198,2022-01-24 03:38:30,4.744,1570776534,1977018578,4.383708,30.0,0.000000,0.000000,0.000000,...,0,0,0,1,0,84,10,24,-5,Clear
4,6198,2022-01-24 03:38:30,4.744,1977018580,1977018576,24.195593,31.0,30.725686,46.437680,30.725999,...,80,0,0,1,0,84,10,24,-5,Clear


In [19]:
big_table_test_df.to_csv("datasets/big-table-test.csv", index=False)